In [58]:
#import the needed libraries
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os

from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics


from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV



In [59]:
#get and clean stats datset
stats = pd.read_csv("/home/lemina/Desktop/data/stats.csv")
stats_clean = stats.dropna()
stats_clean.columns
#we will use stats.csv to determine some information about the performance of every player in the previous matches
del stats_clean['Unnamed: 0']
del stats_clean['MT']
stats_clean

,ID1,ID2,ID_T,ID_R,FS_1,FSOF_1,ACES_1,DF_1,UE_1,W1S_1,...,NA_2,NAOF_2,TPW_2,FAST_2,A1S_2,A2S_2,RPW_1,RPWOF_1,RPW_2,RPWOF_2
312,7042,2992,8885,1,36.0,43.0,3.0,3.0,11.0,29,...,7.0,19.0,30,208.0,188.0,150.0,27.0,46.0,11.0,43.0
314,9094,9831,8885,1,63.0,107.0,6.0,4.0,47.0,43,...,11.0,17.0,95,198.0,177.0,140.0,34.0,88.0,41.0,107.0
315,8146,603,8885,1,62.0,84.0,12.0,5.0,43.0,48,...,15.0,25.0,83,187.0,162.0,134.0,38.0,95.0,26.0,84.0
323,1003,9842,8885,1,65.0,91.0,3.0,2.0,43.0,44,...,13.0,30.0,85,202.0,170.0,122.0,46.0,96.0,35.0,91.0
326,47,778,8885,1,52.0,87.0,3.0,6.0,13.0,43,...,29.0,56.0,80,207.0,189.0,164.0,21.0,78.0,23.0,87.0
330,9833,6225,8885,1,29.0,49.0,6.0,3.0,11.0,28,...,14.0,28.0,50,221.0,187.0,148.0,24.0,66.0,8.0,49.0
331,1837,7636,8885,1,66.0,83.0,0.0,2.0,30.0,41,...,19.0,32.0,74,187.0,172.0,145.0,37.0,78.0,33.0,83.0
334,3833,869,8885,1,78.0,125.0,4.0,0.0,70.0,50,...,17.0,26.0,123,187.0,162.0,130.0,55.0,125.0,53.0,125.0
336,714,1621,8885,1,41.0,68.0,3.0,2.0,15.0,26,...,11.0,25.0,67,197.0,180.0,149.0,36.0,75.0,28.0,68.0
340,3676,12896,8885,1,81.0,144.0,12.0,3.0,40.0,59,...,13.0,20.0,135,216.0,194.0,147.0,44.0,131.0,48.0,144.0


In [60]:
#look how many times a winner player (colomn ID1 ) had won in the past

from pandas import *
winners =stats_clean['ID1'].value_counts()

df=winners.to_frame(name="count_Won").reset_index()

winnersDF = df.rename(columns={'index': 'ID'})
winnersDF

,ID,count_Won
0,5992,97
1,1075,76
2,19,68
3,673,55
4,831,54
5,677,50
6,1092,48
7,3985,46
8,674,40
9,11704,39


In [61]:
#look how many times a looser player (colomn ID2 ) had lost in the past
loosers =stats_clean['ID2'].value_counts()
df=loosers.to_frame(name="count_Loss").reset_index()
loosersDF = df.rename(columns={'index': 'ID'})
loosersDF

,ID,count_Loss
0,831,17
1,7,16
2,11522,16
3,7043,16
4,7459,16
5,11704,16
6,4010,15
7,674,15
8,1092,15
9,5837,15


In [62]:
#merge so that we can find for ever player  how many  times  he won and lost 
result = pd.merge(loosersDF,winnersDF,on=['ID'])

result


,ID,count_Loss,count_Won
0,831,17,54
1,7,16,13
2,11522,16,21
3,7043,16,14
4,7459,16,21
5,11704,16,39
6,4010,15,26
7,674,15,40
8,1092,15,48
9,5837,15,28


In [63]:
#here we consider every winner and look for the average of  their good breaks out of the breaks he had
stats_clean['wonBreaks'] = stats_clean['RPW_1']/stats_clean['RPWOF_1']*100
dfWon = stats_clean[['ID1','wonBreaks']] 
dfWon = dfWon.rename(columns={'ID1': 'ID'})
avg_wonBreak = dfWon['wonBreaks'].groupby(dfWon['ID']).mean()
avg_wonBreak

frame7=avg_wonBreak.to_frame(name="avg_wonBreakW").reset_index()

frame7

,ID,avg_wonBreakW
0,7,42.142244
1,10,50.925926
2,12,38.857143
3,19,43.426722
4,22,42.974965
5,47,26.923077
6,75,40.951709
7,78,39.383279
8,84,39.361788
9,89,38.695677


In [64]:
#here we consider every looser and look for the average of  their good breaks out of the breaks he had
stats_clean['wonBreaksL'] = stats_clean['RPW_2']/stats_clean['RPWOF_2']*100
dfLost = stats_clean[['ID2','wonBreaksL']] 
dfLost = dfLost.rename(columns={'ID2': 'ID'})
avg_lostBreak = dfLost['wonBreaksL'].groupby(dfWon['ID']).mean()
avg_lostBreak

frame8=avg_lostBreak.to_frame(name="avg_wonBreakL").reset_index()



#merge so that we know how many good breaks a player had done (in the two  cases whether he won or lost)  

result1 = pd.merge(frame7,frame8,on=['ID'])
result1

,ID,avg_wonBreakW,avg_wonBreakL
0,7,42.142244,34.793196
1,10,50.925926,33.823529
2,12,38.857143,34.969325
3,19,43.426722,25.209276
4,22,42.974965,30.210359
5,47,26.923077,26.436782
6,75,40.951709,32.534982
7,78,39.383279,32.492877
8,84,39.361788,30.016200
9,89,38.695677,26.037860


In [65]:
#look for how many times a winner had won points from the ones he had played 
stats_clean['WonPoints'] = stats_clean['RPW_1']/stats_clean['RPWOF_1']*100


dfWonPoints = stats_clean[['ID1','WonPoints']] 
dfWonPoints = dfWonPoints.rename(columns={'ID1': 'ID'})
avg_Wonpoints_perUser = dfWonPoints['WonPoints'].groupby(dfWonPoints['ID']).mean()
avg_Wonpoints_perUser

frame1=avg_Wonpoints_perUser.to_frame(name="avg_pointsW").reset_index()

frame1

,ID,avg_pointsW
0,7,42.142244
1,10,50.925926
2,12,38.857143
3,19,43.426722
4,22,42.974965
5,47,26.923077
6,75,40.951709
7,78,39.383279
8,84,39.361788
9,89,38.695677


In [67]:
#look for how many times a looser had won points from the ones he had played 
stats_clean['LostPoints'] = stats_clean['RPW_2']/stats_clean['RPWOF_2']*100
dfLostPoints = stats_clean[['ID2','LostPoints']] 
dfLostPoints = dfLostPoints.rename(columns={'ID2': 'ID'})
avg_lostpoints_perUser = dfLostPoints['LostPoints'].groupby(dfLostPoints['ID']).mean()
avg_lostpoints_perUser

frame2=avg_lostpoints_perUser.to_frame(name="avg_pointsL").reset_index()
frame2


,ID,avg_pointsL
0,7,35.745448
1,10,31.405229
2,12,43.220339
3,19,31.542397
4,22,30.468729
5,72,24.671104
6,75,29.965591
7,78,30.058139
8,84,26.216577
9,89,29.778426


In [68]:
#merge to know each player performance: won points whether he won or failed
resultPoints = pd.merge(frame1,frame2,on=['ID'])
resultPoints

,ID,avg_pointsW,avg_pointsL
0,7,42.142244,35.745448
1,10,50.925926,31.405229
2,12,38.857143,43.220339
3,19,43.426722,31.542397
4,22,42.974965,30.468729
5,75,40.951709,29.965591
6,78,39.383279,30.058139
7,84,39.361788,26.216577
8,89,38.695677,29.778426
9,427,45.159058,29.054650


In [69]:
#include this information in the created dataframe
d = pd.merge(result,resultPoints,on=['ID'])
DF = pd.merge(d,result1,on=['ID'])
DF

,ID,count_Loss,count_Won,avg_pointsW,avg_pointsL,avg_wonBreakW,avg_wonBreakL
0,831,17,54,42.015088,29.688267,42.015088,28.298502
1,7,16,13,42.142244,35.745448,42.142244,34.793196
2,11522,16,21,35.161828,27.257577,35.161828,30.387566
3,7043,16,14,39.951653,29.087182,39.951653,29.913653
4,7459,16,21,38.046669,28.133013,38.046669,26.886547
5,11704,16,39,37.648623,25.334035,37.648623,25.019130
6,4010,15,26,44.423721,33.562129,44.423721,33.814487
7,674,15,40,42.396748,29.555993,42.396748,29.063958
8,1092,15,48,38.947152,32.966179,38.947152,29.336635
9,5837,15,28,33.753310,24.361028,33.753310,22.728627


In [70]:
#caluclate the averages of double faults and direct errors of every winner
stats_clean['FaultsW'] = (stats_clean['DF_1']+stats_clean['UE_1']).mean()
faults1 = stats_clean[['ID1','FaultsW']] 
faults1 = faults1.rename(columns={'ID1': 'ID'})
faults_perUser =faults1['FaultsW'].groupby(faults1['ID']).mean()
faults_perUser

frame5=faults_perUser.to_frame(name="avgFaultsW").reset_index()
frame5


,ID,avgFaultsW
0,7,33.3703
1,10,33.3703
2,12,33.3703
3,19,33.3703
4,22,33.3703
5,47,33.3703
6,75,33.3703
7,78,33.3703
8,84,33.3703
9,89,33.3703


In [15]:
#caluclate the averages of dauble faults and direct errors of every looser
stats_clean['FaultsL'] = stats_clean['DF_2']+stats_clean['UE_2']
faults2 = stats_clean[['ID2','FaultsL']] 
faults2 = faults2.rename(columns={'ID2': 'ID'})
faultsl_perUser =faults2['FaultsL'].groupby(faults2['ID']).mean()
faultsl_perUser

frame6=faultsl_perUser.to_frame(name="avgFaultsL").reset_index()



In [71]:
#merge to know how many errors a player  had done (in both cases win/lose)
FaultPoints = pd.merge(frame5,frame6,on=['ID'])
FaultPoints

,ID,avgFaultsW,avgFaultsL
0,7,33.3703,52.250000
1,10,33.3703,26.500000
2,12,33.3703,37.000000
3,19,33.3703,45.857143
4,22,33.3703,46.900000
5,75,33.3703,35.800000
6,78,33.3703,31.800000
7,84,33.3703,25.222222
8,89,33.3703,32.750000
9,427,33.3703,30.666667


In [72]:
#include this new calculated feature
Data = pd.merge(DF,FaultPoints,on=['ID'])
Data

,ID,count_Loss,count_Won,avg_pointsW,avg_pointsL,avg_wonBreakW,avg_wonBreakL,avgFaultsW,avgFaultsL
0,831,17,54,42.015088,29.688267,42.015088,28.298502,33.3703,43.176471
1,7,16,13,42.142244,35.745448,42.142244,34.793196,33.3703,52.250000
2,11522,16,21,35.161828,27.257577,35.161828,30.387566,33.3703,31.625000
3,7043,16,14,39.951653,29.087182,39.951653,29.913653,33.3703,43.750000
4,7459,16,21,38.046669,28.133013,38.046669,26.886547,33.3703,39.125000
5,11704,16,39,37.648623,25.334035,37.648623,25.019130,33.3703,50.687500
6,4010,15,26,44.423721,33.562129,44.423721,33.814487,33.3703,41.866667
7,674,15,40,42.396748,29.555993,42.396748,29.063958,33.3703,29.800000
8,1092,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333
9,5837,15,28,33.753310,24.361028,33.753310,22.728627,33.3703,47.066667


In [18]:

#here we consider every winner and look for the average of  their good serves out of the serves he played 

stats_clean['goodServe'] = stats_clean['FS_1']/stats_clean['FSOF_1']*100


dfgoodServe = stats_clean[['ID1','goodServe']] 
dfgoodServe= dfgoodServe.rename(columns={'ID1': 'ID'})

avg_goodServe_perUser = dfgoodServe['goodServe'].groupby(dfgoodServe['ID']).mean()
avg_goodServe_perUser 
frame3=avg_goodServe_perUser.to_frame(name="avg_goodServeW").reset_index()


In [19]:
#here we consider every looser and look for the average of  their good serves out of the serves he played
stats_clean['lostServe'] = stats_clean['FS_2']/stats_clean['FSOF_2']*100
dfLost = stats_clean[['ID2','lostServe']] 
dfLost = dfLost.rename(columns={'ID2': 'ID'})
avg_lostServe = dfLost['lostServe'].groupby(dfLost['ID']).mean()
avg_lostServe

frame4=avg_lostServe.to_frame(name="avg_goodServeL").reset_index()



In [20]:
#we merge to see for every player the % of his good serves, whether winner or looser
resultServe = pd.merge(frame3,frame4,on=['ID'])
resultServe

In [73]:
#we add this calculated feature to our final created statistical dataset
finalDataSet = pd.merge(Data,resultServe,on=['ID'])
finalDataSet

#these statistics were calculated from the colomns of stats.csv 

#for each player : 
 #   count_loss : how many times he lost in the past
  #  count_won: how many times he won in the past
  # avg_pointsW :  the average of points got when he won
  # avg_pointsL : the average of points got when he lost   
  # avg_wonBreakW : the average of good breaks he made when he won
  # avg_wonBreakL : the average of good breaks he made when he lost  
  # avgFaultsW : average of faults made when he won 
  # avgFaultsL : average of faults made when lost 
  # avg_goodServeW :   average of good serves from the total serves he made when he won 
  # avg_goodServeL : average of good serves from the total serves he made when he lost the match    



,ID,count_Loss,count_Won,avg_pointsW,avg_pointsL,avg_wonBreakW,avg_wonBreakL,avgFaultsW,avgFaultsL,avg_goodServeW,avg_goodServeL
0,831,17,54,42.015088,29.688267,42.015088,28.298502,33.3703,43.176471,58.043741,59.224293
1,7,16,13,42.142244,35.745448,42.142244,34.793196,33.3703,52.250000,52.946974,53.071159
2,11522,16,21,35.161828,27.257577,35.161828,30.387566,33.3703,31.625000,68.678388,68.794538
3,7043,16,14,39.951653,29.087182,39.951653,29.913653,33.3703,43.750000,63.367830,59.872162
4,7459,16,21,38.046669,28.133013,38.046669,26.886547,33.3703,39.125000,66.966046,65.664520
5,11704,16,39,37.648623,25.334035,37.648623,25.019130,33.3703,50.687500,63.972431,62.232387
6,4010,15,26,44.423721,33.562129,44.423721,33.814487,33.3703,41.866667,59.962194,59.705990
7,674,15,40,42.396748,29.555993,42.396748,29.063958,33.3703,29.800000,61.819808,63.350994
8,1092,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
9,5837,15,28,33.753310,24.361028,33.753310,22.728627,33.3703,47.066667,69.076495,72.922125


In [74]:
#get and clean train.csv
train = pd.read_csv("/home/lemina/Desktop/data/train.csv")
train_clean = train.dropna()

del train_clean['Unnamed: 0']
del train_clean['RESULT_G']



In [75]:
# retrieve information of the winning players from our created statistical dataset

train_clean = train_clean.rename(columns={'ID2_G': 'ID'})

trclean = pd.merge(train_clean,finalDataSet,on=['ID'],how='inner')
trclean= trclean.rename(columns={'ID': 'ID_2'})
trclean.columns
traincl = trclean.rename(columns={'ID1_G': 'ID'})
trainingDataSet = pd.merge(traincl,finalDataSet,on=['ID'],how='inner',suffixes=('_1', '_2'))
trainingDataSet=trainingDataSet.rename(columns={'ID': 'ID_1'})
trainingDataSet.columns


Index([u'ID_1', u'ID_2', u'ID_T_G', u'ID_R_G', u'DATE_G', u'count_Loss_1',
       u'count_Won_1', u'avg_pointsW_1', u'avg_pointsL_1', u'avg_wonBreakW_1',
       u'avg_wonBreakL_1', u'avgFaultsW_1', u'avgFaultsL_1',
       u'avg_goodServeW_1', u'avg_goodServeL_1', u'count_Loss_2',
       u'count_Won_2', u'avg_pointsW_2', u'avg_pointsL_2', u'avg_wonBreakW_2',
       u'avg_wonBreakL_2', u'avgFaultsW_2', u'avgFaultsL_2',
       u'avg_goodServeW_2', u'avg_goodServeL_2'],
      dtype='object')

In [76]:
#our trainingset has column for winner and column for looser 
#we are going to change this to player1 and player2 with the correspondant stats to train the model
# we will devide the training set into two subsets 
#subset A has the winner in first column and looser in second column (like the oringinal dataset) result = 1 (Player1 won)
#subset B has the winner in second column and looser in  first column  result = 2 (player 2 won)
#this will help us make the classification for future challenges (whether 1 or 2 )


subsetA = trainingDataSet[0:len(trainingDataSet.index)/2]
subsetA
subsetA.insert(4,'Result',1)
subsetA = subsetA.rename(columns={'ID_1': 'Player1' , 'ID_2':'Player2'})


subsetB= trainingDataSet[len(trainingDataSet.index)/2:len(trainingDataSet.index)]

subsetB1 = subsetB[['ID_2', 'count_Loss_1','DATE_G','ID_T_G','ID_R_G','count_Won_1','avg_pointsW_1','avg_pointsL_1','avg_wonBreakW_1','avg_wonBreakL_1',
             'avgFaultsW_1','avgFaultsL_1','avg_goodServeW_1','avg_goodServeL_1','ID_1','count_Loss_2','count_Won_2','avg_pointsW_2',
             'avg_pointsL_2','avg_wonBreakW_2','avg_wonBreakL_2','avgFaultsW_2','avgFaultsL_2','avg_goodServeW_2','avg_goodServeL_2']]

subsetB1.insert(4,'Result',2)
subsetB1 = subsetB1.rename(columns={'ID_1': 'Player2' , 'ID_2':'Player1', 'count_Loss_1' : 'count_Loss_2','count_Won_1': 'count_Won_2',
                                   'avg_pointsW_1':'avg_pointsW_2','avg_pointsL_1':'avg_pointsL_2','avg_wonBreakW_1':'avg_wonBreakW_2',
                                   'avg_wonBreakL_1':'avg_wonBreakL_2','avgFaultsW_1':'avgFaultsW_2','avgFaultsL_1':'avgFaultsL_2',
                                   'avg_goodServeW_1':'avg_goodServeW_2','avg_goodServeL_1':'avg_goodServeL_2','count_Loss_2':'count_Loss_1','count_Won_2':'count_Won_1',
                                   'avg_pointsW_2':'avg_pointsW_1',
             'avg_pointsL_2':'avg_pointsL_1','avg_wonBreakW_2':'avg_wonBreakW_1','avg_wonBreakL_2':'avg_wonBreakL_1','avgFaultsW_2':'avgFaultsW_1','avgFaultsL_2':'avgFaultsL_1','avg_goodServeW_2':'avg_goodServeW_1','avg_goodServeL_2':'avg_goodServeL_1'})
subsetB1

frames = [subsetA, subsetB1]
Trainingset = pd.concat(frames)
Trainingset.drop_duplicates(inplace = True)
#we get out final TrainingSet
TrainingSet =Trainingset.reset_index(drop=True)


In [77]:
#get and clean test.csv 
test = pd.read_csv("/home/lemina/Desktop/data/test.csv")
test_clean = test.dropna()


In [78]:
# retrieve information of the first players ID1 from our created statistical dataset
#this will make the same dataframe structure for training and test datasets

test_clean = test_clean.rename(columns={'ID2_G': 'ID'})

clean = pd.merge(test_clean,finalDataSet,on=['ID'],how='inner')
clean= clean.rename(columns={'ID': 'ID_2'})
clean.columns
clt = clean.rename(columns={'ID1_G': 'ID'})
testDataSet = pd.merge(clt,finalDataSet,on=['ID'],how='inner',suffixes=('_1', '_2'))
testDataSet=testDataSet.rename(columns={'ID': 'ID_1'})
testDataSet.drop_duplicates(inplace = True)
testDataSet.columns


Index([u'ID_1', u'ID_2', u'ID_T_G', u'ID_R_G', u'DATE_G', u'count_Loss_1',
       u'count_Won_1', u'avg_pointsW_1', u'avg_pointsL_1', u'avg_wonBreakW_1',
       u'avg_wonBreakL_1', u'avgFaultsW_1', u'avgFaultsL_1',
       u'avg_goodServeW_1', u'avg_goodServeL_1', u'count_Loss_2',
       u'count_Won_2', u'avg_pointsW_2', u'avg_pointsL_2', u'avg_wonBreakW_2',
       u'avg_wonBreakL_2', u'avgFaultsW_2', u'avgFaultsL_2',
       u'avg_goodServeW_2', u'avg_goodServeL_2'],
      dtype='object')

In [79]:
#rename ID_1 and ID_2 by Player1 and Player2 
testDataSet = testDataSet.rename(columns={'ID_1': 'Player1' , 'ID_2':'Player2'})

testDataSet


,Player1,Player2,ID_T_G,ID_R_G,DATE_G,count_Loss_1,count_Won_1,avg_pointsW_1,avg_pointsL_1,avg_wonBreakW_1,...,count_Loss_2,count_Won_2,avg_pointsW_2,avg_pointsL_2,avg_wonBreakW_2,avg_wonBreakL_2,avgFaultsW_2,avgFaultsL_2,avg_goodServeW_2,avg_goodServeL_2
0,1092,20648,12814,5,2016-05-25 00:00:00,3,1,46.938776,25.896722,46.938776,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
1,1092,4146,12812,10,2016-05-20 00:00:00,7,5,33.350153,25.268781,33.350153,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
2,1092,4146,12814,4,2016-05-23 00:00:00,7,5,33.350153,25.268781,33.350153,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
3,1092,7295,12814,7,2016-05-29 00:00:00,10,12,41.109417,29.123779,41.109417,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
4,1092,14432,12812,9,2016-05-19 00:00:00,6,2,48.356060,27.500674,48.356060,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
5,1092,12043,12811,5,2016-05-10 00:00:00,13,10,40.974546,32.232779,40.974546,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
6,1092,7043,12814,6,2016-05-27 00:00:00,16,14,39.951653,29.087182,39.951653,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
7,1092,6101,12812,12,2016-05-21 00:00:00,11,35,39.093551,32.047325,39.093551,...,15,48,38.947152,32.966179,38.947152,29.336635,33.3703,54.733333,57.493976,55.892945
8,1042,20648,12821,4,2016-06-28 00:00:00,3,1,46.938776,25.896722,46.938776,...,11,6,41.304166,33.399240,41.304166,28.978319,33.3703,39.818182,73.582841,71.041424
9,1042,1092,12811,6,2016-05-12 00:00:00,15,48,38.947152,32.966179,38.947152,...,11,6,41.304166,33.399240,41.304166,28.978319,33.3703,39.818182,73.582841,71.041424


In [80]:
#we will use random forest

#we specify our explonatory variables
predictorsTrain =  TrainingSet[['count_Loss_1','count_Won_1','avg_pointsW_1','avg_pointsL_1',
'avg_wonBreakW_1','avg_wonBreakL_1','avgFaultsW_1','avgFaultsL_1','avg_goodServeW_1','avg_goodServeL_1','count_Loss_2','count_Won_2',
'avg_pointsW_2','avg_pointsL_2','avg_wonBreakW_2','avg_wonBreakL_2','avgFaultsW_2','avgFaultsL_2','avg_goodServeW_2','avg_goodServeL_2']]

#we set our target variable
targetsTrain = TrainingSet.Result

# 60% of the data will be considered for trainin and 40% for validating
pred_train, pred_test, tar_train, tar_test  = train_test_split(predictorsTrain, targetsTrain, test_size=.4 ,random_state=123)

#this will be used testDataSet to predict future matches
predictorsTest =  testDataSet[['count_Loss_1','count_Won_1','avg_pointsW_1','avg_pointsL_1',
'avg_wonBreakW_1','avg_wonBreakL_1','avgFaultsW_1','avgFaultsL_1','avg_goodServeW_1','avg_goodServeL_1','count_Loss_2','count_Won_2',
'avg_pointsW_2','avg_pointsL_2','avg_wonBreakW_2','avg_wonBreakL_2','avgFaultsW_2','avgFaultsL_2','avg_goodServeW_2','avg_goodServeL_2']]




In [81]:
#use grid search to look for best parameters to fit the model


classifier=RandomForestClassifier( n_jobs=-1,max_features= 'sqrt' , oob_score = True ) 

param_grid = { 
           "n_estimators" : [18, 27, 36, 45,54],
           "max_depth" : [10, 15, 20, 25, 30],
           "min_samples_leaf" : [4, 6, 8, 10]}
           

CV_classifier = GridSearchCV(estimator=classifier, param_grid=param_grid, cv= 8)
CV_classifier.fit(pred_train,tar_train) 
print CV_classifier.best_params_


{'n_estimators': 54, 'max_depth': 25, 'min_samples_leaf': 6}


In [82]:
# we find {'n_estimators': 54, 'max_depth': 20, 'min_samples_leaf': 6}
# we fit the model with these hyperparamaeters
rfclassifier = RandomForestClassifier(n_estimators=54, max_depth=25, max_features='sqrt', min_samples_leaf = 6)
rfclassifier=rfclassifier.fit(pred_train,tar_train)
preds=rfclassifier.predict(pred_test)
preds
sklearn.metrics.confusion_matrix(tar_test,preds)
sklearn.metrics.accuracy_score(tar_test, preds)
# the model accuracy on the validation set is 84% / 84% were classified correctly 
#now we will use this classifier to predict future games 

predictions=rfclassifier.predict(predictorsTest)
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,
       1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [83]:
unique, counts = np.unique(predictions, return_counts=True)
dict(zip(unique, counts))

{1: 285, 2: 347}

In [84]:
model = ExtraTreesClassifier()
model.fit(pred_train,tar_train)
# display the relative importance of each attribute
print(model.feature_importances_)

[0.04393167 0.05468092 0.04853466 0.04292061 0.04646339 0.04314437
 0.         0.04502655 0.04646371 0.04986486 0.1391868  0.12049758
 0.04886814 0.05134543 0.04430801 0.05045895 0.         0.04332387
 0.04321349 0.03776699]


In [85]:
winner = np.asarray(predictions)
Winner= pd.DataFrame(winner, columns=['Winner'])
Winner

,Winner
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [86]:
Test=testDataSet[['Player1','Player2','ID_T_G','ID_R_G','DATE_G']]
Test 

,Player1,Player2,ID_T_G,ID_R_G,DATE_G
0,1092,20648,12814,5,2016-05-25 00:00:00
1,1092,4146,12812,10,2016-05-20 00:00:00
2,1092,4146,12814,4,2016-05-23 00:00:00
3,1092,7295,12814,7,2016-05-29 00:00:00
4,1092,14432,12812,9,2016-05-19 00:00:00
5,1092,12043,12811,5,2016-05-10 00:00:00
6,1092,7043,12814,6,2016-05-27 00:00:00
7,1092,6101,12812,12,2016-05-21 00:00:00
8,1042,20648,12821,4,2016-06-28 00:00:00
9,1042,1092,12811,6,2016-05-12 00:00:00


In [87]:
TestSet =Test.join(Winner)
TestSet.columns
TestSet.rename(columns={'0': 'Winner'})
TestSet.columns
TestSet

,Player1,Player2,ID_T_G,ID_R_G,DATE_G,Winner
0,1092,20648,12814,5,2016-05-25 00:00:00,1
1,1092,4146,12812,10,2016-05-20 00:00:00,1
2,1092,4146,12814,4,2016-05-23 00:00:00,1
3,1092,7295,12814,7,2016-05-29 00:00:00,1
4,1092,14432,12812,9,2016-05-19 00:00:00,1
5,1092,12043,12811,5,2016-05-10 00:00:00,1
6,1092,7043,12814,6,2016-05-27 00:00:00,1
7,1092,6101,12812,12,2016-05-21 00:00:00,1
8,1042,20648,12821,4,2016-06-28 00:00:00,1
9,1042,1092,12811,6,2016-05-12 00:00:00,1


In [88]:
TestSet.to_csv('/home/lemina/Desktop/test.csv', sep=',', encoding='utf-8')